# Examination dynamics

In [1]:
import sys
from pathlib import Path
sys.path.insert(1, '../../src/SEIR/')
sys.path.insert(1, '../../src/utils/')

from datetime import datetime
import numpy as np
from numpy import linalg as LA
import pandas as pd
from time import time
import multiprocessing
from joblib import Parallel, delayed

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows': 
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')


from class_SEIRQ import SEIR
from Quarantine import Quarantine
from Quarantine import Exams

Linux


### Parameters

In [2]:
beta = 0.15 # Contagion rate
mu = 0 # E0/I0 initial rate
sigma = 0.2
gamma = 0.1

SeroPrevalence = 1
expinfection = 0

# Simulation time
tsim = 1000
# Population
population = 100000
# Initial Active Infected 
I0 = 100
I_ac0 = 100

# Quarantine
s1 = Quarantine(1)

### Exam Dynamics

In [3]:
duty = 1
period = 1
testaccuracy=0.9
lambda_Tr = 0 # traceability
lambda_Q=1 # Proportion of effectively quarantined detected infected
T_Q = 14 # Time in quarantine

# Exam Rate
examrate = np.arange(1,20,1)*population/100# Persons per day

# Delay Time
T_T = [1,2,3,4,5]

psi = [Exams(i,period,duty) for i in examrate] 

In [4]:
len(examrate)

19

## Simulate

In [5]:
# Define Simulations
sims = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(T_T)):
        aux.append(SEIR(tsim=tsim,alpha=s1.alpha,psi = psi[i],beta=beta,sigma=sigma,gamma=gamma,mu=mu,I0=I0,population=population,expinfection=0,testaccuracy=testaccuracy,lambda_Q=lambda_Q,T_T = T_T[j],T_Q = T_Q,lambda_Tr=lambda_Tr))
    sims.append(aux)

In [7]:
#%%capture
# To show the cell's output comment the former line

# For parallel simulation
def simulate(simulation,j,tsim):
    simulation[j].integr_sci(0,tsim,0.1)
    return simulation[j]

# Reference simulation
simulation_reference =  SEIR(tsim=tsim,alpha=s1.alpha,psi = 0,beta=beta,mu=mu,sigma=sigma,gamma=gamma,I0=I0,population=population,expinfection=0,SeroPrevFactor=1)
simulation_reference.integr_sci(0,tsim,0.01)
print('simulation finished')

# Run simulation
num_cores = multiprocessing.cpu_count()
simulation = []
for i in range(len(examrate)):
    simulation.append(Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sims[i],j,tsim) for j in range(len(sims[i]))))

print('ready')

simulation finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    1.0s
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    1.0s remaining:    1.5s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    1.0s remaining:    0.7s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    1.0s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.7s
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.7s remaining:    1.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.7s remaining:    0.5s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.7s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent worker

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0443s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0508s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
ready

## Analysis

In [8]:
# Calculate indicators
peak = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(T_T)):
        aux.append(100*simulation[i][j].peak/population)
    peak.append(aux)

    
peaktime = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(T_T)):
        aux.append(simulation[i][j].peak_t)
    peaktime.append(aux)

prevalence = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(T_T)):    
        aux.append(simulation[i][j].prevalence_total[-1])  
    prevalence.append(aux)
    
    
# Calculate indicators
peakvsnoexam = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(T_T)):
        aux.append(100*simulation[i][j].peak/simulation_reference.peak)
    peakvsnoexam.append(aux)
    
# Calculate indicators
totinfected = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(T_T)):
        aux.append(100*simulation[i][j].I_ac[-1]/simulation_reference.I_ac[-1])
    totinfected.append(aux)    

### Grid Plot

In [9]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(T_T)))
n = 4
fig, axs = plt.subplots(int(len(examrate)/n)+(int(len(examrate))%n > 0)*1, n)
for i in range(len(examrate)):
    axs[int(i/n),i%n].plot(simulation_reference.t,simulation_reference.I_d,label='I_d No exams',color = 'black',linestyle='dashed')
    for j in range(len(T_T)):
        axs[int(i/n),i%n].plot(simulation[i][j].t,simulation[i][j].I_d,label='Examination results delay: '+str(T_T[j]))
        axs[int(i/n),i%n].legend(loc=0)
    axs[int(i/n),i%n].set_title('Exam Rate: '+str(examrate[i]/population*100)+'%')
fig.suptitle('New Daily Infected')

Text(0.5, 0.98, 'New Daily Infected')

In [10]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(T_T)))
n = 4
fig, axs = plt.subplots(int(len(examrate)/n)+(int(len(examrate))%n > 0)*1, n)
for i in range(len(examrate)):
    axs[int(i/n),i%n].plot(simulation_reference.t,simulation_reference.I_T,label='I_d No exams',color = 'black',linestyle='dashed')
    for j in range(len(T_T)):
        axs[int(i/n),i%n].plot(simulation[i][j].t,simulation[i][j].I_T,label='Examination results delay: '+str(T_T[j]))
        axs[int(i/n),i%n].legend(loc=0)
    axs[int(i/n),i%n].set_title('Exam Rate: '+str(examrate[i]))
fig.suptitle('Infected waiting for test results')
fig.show()

In [11]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(T_T,examrate*100/population,peakvsnoexam) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Peak Size % respect to no exams scenario')
ax.set_xlabel('Results Delay (days)')
ax.set_ylabel('Normalized (%) Exams Rate')
plt.show() 

In [ ]:
np.min(totinfected)


In [ ]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(T_T,examrate*100/population,totinfected,10) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Peak Size % respect to no exams scenario')
ax.set_xlabel('Results Delay (days)')
ax.set_ylabel('Normalized (%) Exams Rate')
plt.show() 

In [ ]:
totinfected

In [ ]:
examrate/population*100

# Delay Days Analysis

In [ ]:
for i in range(len(examrate)):
    plt.plot(T_T,peakvsnoexam[i],label='Exam Rate: '+str(examrate[i]))
plt.title('Exam results delay effects: Peak Size respect to no exam application')
plt.legend(loc=0)
plt.show()

In [ ]:
for i in range(len(examrate)):
    plt.plot(T_T,peak[i]/peak[i][0],label='Exam Rate: '+str(examrate[i]))
plt.title('Exam results delay effects: Peak Size respect to minimum delay')
plt.legend(loc=0)
plt.show()

# Beta Analysis

In [ ]:
mu = 0 # E0/I0 initial rate
sigma = 0.2
gamma = 0.1

SeroPrevalence = 1
expinfection = 0

# Simulation time
tsim = 1000
# Population
population = 100000
# Initial Active Infected 
I0 = 100
I_ac0 = 100

# Quarantine
s1 = Quarantine(1)

In [ ]:
duty = 1
period = 1
testaccuracy=0.9
lambda_Tr = 0 # traceability
lambda_Q=1 # Proportion of effectively quarantined detected infected
T_Q = 14 # Time in quarantine

# Exam Rate
examrate = np.arange(1,20,1)*population/100# Persons per day
# Delay Time
T_T = 2
beta = np.arange(0.1,0.2,0.01)
psi = [Exams(i,period,duty) for i in examrate] 

In [ ]:
# Define Simulations
sims_reference = []
for i in range(len(beta)):
    sims_reference.append(SEIR(tsim=tsim,alpha=s1.alpha,psi = 0,beta=beta[i],sigma=sigma,gamma=gamma,mu=mu,I0=I0,population=population,expinfection=0,testaccuracy=testaccuracy,lambda_Q=lambda_Q,T_T = T_T,T_Q = T_Q,lambda_Tr=lambda_Tr))

In [ ]:
# For parallel simulation
def simulate_reference(simulation,tsim):
    simulation.integr_sci(0,tsim,0.1)
    return simulation

# Reference simulation
# Run simulation
num_cores = multiprocessing.cpu_count()
simulation_reference = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate_reference)(sims_reference[i],tsim) for i in range(len(beta)))

print('simulation finished')


In [ ]:
# Define Simulations
sims = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux.append(SEIR(tsim=tsim,alpha=s1.alpha,psi = psi[i],beta=beta[j],sigma=sigma,gamma=gamma,mu=mu,I0=I0,population=population,expinfection=0,testaccuracy=testaccuracy,lambda_Q=lambda_Q,T_T = T_T,T_Q = T_Q,lambda_Tr=lambda_Tr))
    sims.append(aux)

In [ ]:
%%capture
# To show the cell's output comment the former line
# For parallel simulation
def simulate(simulation,j,tsim):
    simulation[j].integr_sci(0,tsim,0.1)
    return simulation[j]

# Run simulation
num_cores = multiprocessing.cpu_count()
simulation = []
for i in range(len(examrate)):
    simulation.append(Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sims[i],j,tsim) for j in range(len(sims[i]))))

print('ready')

In [ ]:
# Calculate indicators
peak = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux.append(100*simulation[i][j].peak/population)
    peak.append(aux)

    
peaktime = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux.append(simulation[i][j].peak_t)
    peaktime.append(aux)

prevalence = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):    
        aux.append(simulation[i][j].prevalence_total[-1])  
    prevalence.append(aux)
    
    
# Calculate indicators
peakvsnoexam = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux.append(100*simulation[i][j].peak/simulation_reference[j].peak)
    peakvsnoexam.append(aux)
# Calculate indicators
totinfected = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux.append(100*simulation[i][j].I_ac[-1]/simulation_reference[j].I_ac[-1])
    totinfected.append(aux)  

### Grid Plot

In [ ]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(beta)))
n = 4
fig, axs = plt.subplots(int(len(examrate)/n)+(int(len(examrate))%n > 0)*1, n)
for i in range(len(examrate)):
    axs[int(i/n),i%n].plot(simulation_reference.t,simulation_reference.I_d,label='I_d No exams',color = 'black',linestyle='dashed')
    for j in range(len(beta)):
        axs[int(i/n),i%n].plot(simulation[i][j].t,simulation[i][j].I_d,label='Beta: '+str(beta[j]))
        axs[int(i/n),i%n].legend(loc=0)
    axs[int(i/n),i%n].set_title('Exam Rate: '+str(examrate[i]/population*100)+'%')
fig.suptitle('New Daily Infected')

In [ ]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(T_T)))
n = 4
fig, axs = plt.subplots(int(len(examrate)/n)+(int(len(examrate))%n > 0)*1, n)
for i in range(len(examrate)):
    axs[int(i/n),i%n].plot(simulation_reference.t,simulation_reference.I_T,label='I_d No exams',color = 'black',linestyle='dashed')
    for j in range(len(T_T)):
        axs[int(i/n),i%n].plot(simulation[i][j].t,simulation[i][j].I_T,label='Examination results delay: '+str(T_T[j]))
        axs[int(i/n),i%n].legend(loc=0)
    axs[int(i/n),i%n].set_title('Exam Rate: '+str(examrate[i]))
fig.suptitle('Infected waiting for test results')
fig.show()

In [ ]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(beta,examrate*100/population,totinfected,20) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Total Infected respect to no exams scenario')
ax.set_xlabel('beta')
ax.set_ylabel('Normalized (%) Exams Rate')
plt.show() 

# Beta, delay and exam rate
exam rate i  
beta j  
delay days k  


In [2]:
mu = 0 # E0/I0 initial rate
sigma = 0.2
gamma = 0.1

SeroPrevalence = 1
expinfection = 0

# Simulation time
tsim = 1000
# Population
population = 100000
# Initial Active Infected 
I0 = 100
I_ac0 = 100

# Quarantine
s1 = Quarantine(1)

In [78]:
duty = 1
period = 1
testaccuracy=0.9
lambda_Tr = 0 # traceability
lambda_Q=1 # Proportion of effectively quarantined detected infected
T_Q = 14 # Time in quarantine

# Exam Rate
examrate = np.arange(1,21,1)*population/100# Persons per day

# Delay Time
T_T = [1,2,3,4,5]
# Beta
beta = np.arange(0.11,0.21,0.01)

psi = [Exams(i,period,duty) for i in examrate] 

In [79]:
# Define Simulations
sims_reference = []
for i in range(len(beta)):
    sims_reference.append(SEIR(tsim=tsim,alpha=s1.alpha,psi = 0,beta=beta[i],sigma=sigma,gamma=gamma,mu=mu,I0=I0,population=population,expinfection=0,testaccuracy=testaccuracy,lambda_Q=lambda_Q,T_T = 1,T_Q = T_Q,lambda_Tr=lambda_Tr))

In [80]:
# For parallel simulation
def simulate_reference(simulation,tsim):
    simulation.integr_sci(0,tsim,0.1)
    return simulation

# Reference simulation
# Run simulation
num_cores = multiprocessing.cpu_count()
simulation_reference = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate_reference)(sims_reference[i],tsim) for i in range(len(beta)))

print('simulation finished')


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.9s
[Parallel(n_jobs=12)]: Done   2 out of  10 | elapsed:    1.1s remaining:    4.3s
[Parallel(n_jobs=12)]: Done   3 out of  10 | elapsed:    1.2s remaining:    2.7s
[Parallel(n_jobs=12)]: Done   4 out of  10 | elapsed:    1.3s remaining:    1.9s
[Parallel(n_jobs=12)]: Done   5 out of  10 | elapsed:    1.3s remaining:    1.3s
[Parallel(n_jobs=12)]: Done   6 out of  10 | elapsed:    1.3s remaining:    0.9s
[Parallel(n_jobs=12)]: Done   7 out of  10 | elapsed:    1.3s remaining:    0.6s
[Parallel(n_jobs=12)]: Done   8 out of  10 | elapsed:    1.4s remaining:    0.3s
[Parallel(n_jobs=12)]: Done  10 out of  10 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=12)]: Done  10 out of  10 | elapsed:    1.4s finished
simulation finished


In [81]:
# Define Simulations
sims = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux2 = []
        for k in range(len(T_T)):
            aux2.append(SEIR(tsim=tsim,alpha=s1.alpha,psi = psi[i],beta=beta[j],sigma=sigma,gamma=gamma,mu=mu,I0=I0,population=population,expinfection=0,testaccuracy=testaccuracy,lambda_Q=lambda_Q,T_T = T_T[k],T_Q = T_Q,lambda_Tr=lambda_Tr))
        aux.append(aux2)
    sims.append(aux)

In [82]:
#%%capture
# To show the cell's output comment the former line
# For parallel simulation
def simulate(simulation,k,tsim):
    simulation[k].integr_sci(0,tsim,0.1)
    return simulation[k]

# Run simulation
num_cores = multiprocessing.cpu_count()
simulation = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux.append(Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sims[i][j],k,tsim) for k in range(len(sims[i][j]))))
    simulation.append(aux)
print('ready')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0625s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.6s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0364s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0603s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0636s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Para

[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0556s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0514s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0475s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0375s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.0s finished
[Para

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0497s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0488s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Para

...

[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0625s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0352s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel

[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0375s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0440s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0530s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0683s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Para

[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0504s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0509s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0467s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0658s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Para

In [121]:
# Calculate indicators
peak = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux2 = []
        for k in range(len(T_T)):
            aux2.append(100*simulation[i][j][k].peak/population)
        aux.append(aux)
    peak.append(aux)

    
peaktime = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux2 = []
        for k in range(len(T_T)):
            aux2.append(simulation[i][j][k].peak_t)
        aux.append(aux2)
    peaktime.append(aux)

prevalence = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux2 = []
        for k in range(len(T_T)):        
            aux2.append(simulation[i][j][k].prevalence_total[-1])  
        aux.append(aux2)
    prevalence.append(aux)
prevalence = np.array(prevalence)
    
# Calculate indicators
peakvsnoexam = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux2 = []
        for k in range(len(T_T)):        
            aux2.append(100*simulation[i][j][k].peak/simulation_reference[j].peak)
        aux.append(aux2)
    peakvsnoexam.append(aux)
peakvsnoexam = np.array(peakvsnoexam)
# Calculate indicators
totinfected = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux2 = []
        for k in range(len(T_T)):        
            aux2.append(100*simulation[i][j][k].I_ac[-1]/simulation_reference[j].I_ac[-1])
        aux.append(aux2)
    totinfected.append(aux)

totinfected = np.array(totinfected)

# Plots

In [108]:
np.shape(prevalence)

(20, 10, 5)

In [113]:
levels = np.arange(0,105,5)
#levels = [0,5,100]

## Plot 1: Grid Epidemic plots


In [116]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(beta)))
n = 4
j = 4
fig, axs = plt.subplots(int(len(examrate)/n)+(int(len(examrate))%n > 0)*1, n)
for i in range(len(examrate)):
    axs[int(i/n),i%n].plot(simulation_reference[j].t,simulation_reference[j].I_d,label='I_d No exams',color = 'black',linestyle='dashed')
    for k in range(len(T_T)):
        axs[int(i/n),i%n].plot(simulation[i][j][k].t,simulation[i][j][k].I_d,label='Delay days: '+str(T_T[k]))
        axs[int(i/n),i%n].legend(loc=0)
    axs[int(i/n),i%n].set_title('Exam Rate: '+str(round(examrate[i]/population*100))+'%')
fig.suptitle('New Daily Infected')

Text(0.5, 0.98, 'New Daily Infected')

## Plot 2: Tests vs Delay para distintos beta

In [117]:
# Contour Plot
n = 3
fig,ax=plt.subplots(4,3)
for j in range(len(beta)):    
    cp = ax[int(j/n),j%n].contourf(T_T,examrate*100/population,totinfected[:,j,:],levels)     
    ax[int(j/n),j%n].set_title('Beta: '+str(round(beta[j],2)))
    ax[int(j/n),j%n].set_xlabel('Delay days')
    ax[int(j/n),j%n].set_ylabel('Normalized (%) Exams Rate')
    fig.colorbar(cp, ax=ax[int(j/n),j%n])
plt.show() 
#fig.colorbar(cp) # Add a colorbar to a plot
#fig.colorbar(cp, ax=ax)
fig.suptitle('Total infected proportion to the dynamic with no exams')

Text(0.5, 0.98, 'Total infected proportion to the dynamic with no exams')

In [120]:
# Contour Plot
n = 3
fig,ax=plt.subplots(4,3)
for j in range(len(beta)):    
    cp = ax[int(j/n),j%n].contourf(T_T,examrate*100/population,100*np.array(prevalence)[:,j,:],np.arange(0,105,5))
    ax[int(j/n),j%n].set_title('Beta: '+str(round(beta[j],2)))
    ax[int(j/n),j%n].set_xlabel('Delay days')
    ax[int(j/n),j%n].set_ylabel('Normalized (%) Exams Rate')
    fig.colorbar(cp, ax=ax[int(j/n),j%n])
plt.show() 
#fig.colorbar(cp) # Add a colorbar to a plot
#fig.colorbar(cp, ax=ax)
fig.suptitle('Total infected proportion to the dynamic with no exams')

Text(0.5, 0.98, 'Total infected proportion to the dynamic with no exams')

In [ ]:
np.array)prevalence[]

## Plot 3: Test vs Beta para distintos delay

In [118]:
# Contour Plot
n = 3
fig,ax=plt.subplots(2,3)
for k in range(len(T_T)):    
    cp = ax[int(k/n),k%n].contourf(beta,examrate*100/population,totinfected[:,:,k],levels)     
    ax[int(k/n),k%n].set_title('Exams delay: '+str(T_T[k]))
    ax[int(k/n),k%n].set_xlabel('beta')
    ax[int(k/n),k%n].set_ylabel('Normalized (%) Exams Rate')
    fig.colorbar(cp, ax=ax[int(k/n),k%n])
plt.show() 
#fig.colorbar(cp) # Add a colorbar to a plot
#fig.colorbar(cp, ax=ax)
fig.suptitle('Total infected proportion to the dynamic with no exams')

Text(0.5, 0.98, 'Total infected proportion to the dynamic with no exams')

## Plot4: Delay vs beta para distintas tasas de exámenes

In [119]:
# Contour Plot
n = 5
fig,ax=plt.subplots(4,5)
for i in range(len(examrate)):    
    cp = ax[int(i/n),i%n].contourf(T_T,beta,totinfected[i,:,:],levels)     
    ax[int(i/n),i%n].set_title('Exam rate: '+str(100*(examrate[i])/population)+'%')
    ax[int(i/n),i%n].set_xlabel('Delay days')
    ax[int(i/n),i%n].set_ylabel('beta')    
    fig.colorbar(cp, ax=ax[int(i/n),i%n])
plt.show() 
#fig.colorbar(cp) # Add a colorbar to a plot
#fig.colorbar(cp, ax=ax)
fig.suptitle('Total infected proportion to the dynamic with no exams')

Text(0.5, 0.98, 'Total infected proportion to the dynamic with no exams')

## Plot Paper

In [103]:
totinfected[-1]

array([[ 0.47711349,  0.63294227,  0.84410591,  1.14627178,  1.61384922],
       [ 0.32223769,  0.439631  ,  0.60972358,  0.87794451,  1.36195184],
       [ 0.28481356,  0.40209671,  0.58715853,  0.92163202,  1.69887542],
       [ 0.27992607,  0.41244367,  0.64648241,  1.16656516,  3.12934415],
       [ 0.2934401 ,  0.45673748,  0.79398159,  1.85917381, 10.43145576],
       [ 0.32051309,  0.53669286,  1.10064459,  4.53909038, 25.22309423],
       [ 0.36301331,  0.67326635,  1.86682231, 14.27904076, 37.39294846],
       [ 0.42484026,  0.91829709,  4.64881205, 26.19544848, 46.69279338],
       [ 0.51621884,  1.43764188, 13.05607676, 35.97417246, 53.98044273],
       [ 0.65859689,  2.9297631 , 23.04251525, 43.9706888 , 59.80002151]])